# Projet DATA732

ETUDIANTS : RISS Ryan, PRUVOST Jordan

FILLIERE : IDU4

--------------------------------------

## Travail préliminaire (rien de sexuel je vous rassure)

On ajoute les librairies utilisées (notamment Plotly)

In [3]:
import json
import os
import pprint
import plotly.express as px
import pandas as pd

On crée le corpus. Il est composé de 4 .json pour les 4 sites :

- Sputnik (média pro russe)
- PressTV (média pro iranien)
- FDeSouche (média très à droite)
- E&R (pas besoin de préciser "c'est écrit dans son livre" https://www.youtube.com/watch?v=9NmoQ1wohQc)

In [2]:
corpusFiles = ["topaz-data732--all--www.egaliteetreconciliation.fr--20190101--20211231.json", 
"topaz-data732--all--fr.sputniknews.africa--20190101--20211231.json", 
"topaz-data732--all--french.presstv.ir--20190101--20211231.json",
"topaz-data732--all--www.fdesouche.com--20190101--20211231.json"]

## Analyse du nombre d'articles

On crée une fonction nous permettant de connaître le nombre d'articles par mois. On sépare les données selon le site

In [3]:
def getNumberArticlePerMonth(filesNames):
    numberArticlePerMonth = []
    for file in filesNames:
        # open and load file
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        siteName = f.name.split(".")[1]
        years = data["metadata-all"]["fr"]["month"].keys()
        for year in years:
            for month in data["metadata-all"]["fr"]["month"][year].keys():
                num = data["metadata-all"]["fr"]["month"][year][month]["num"]
                numberArticlePerMonth.append({"yearMonth" : (month + "-" + year), "count" : num, "site": siteName})

    #BubbleSort
    good = False

    while(not good):
        good = True
        for i in range(0, len(numberArticlePerMonth) - 1):
            date1 = numberArticlePerMonth[i]["yearMonth"]
            date2 = numberArticlePerMonth[i+1]["yearMonth"]
            (month1, annee1) = date1.split("-")
            (month2, annee2) = date2.split("-")
            if(int(annee1) > int(annee2)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False
            if(annee1 == annee2 and int(month2) < int(month1)):
                numberArticlePerMonth[i], numberArticlePerMonth[i+1] = numberArticlePerMonth[i+1], numberArticlePerMonth[i]
                good = False

    # On crée le bar chart

    fig = px.bar(numberArticlePerMonth, x='yearMonth', y='count', color="site")
    fig.show()

In [4]:
getNumberArticlePerMonth(corpusFiles)

On peut déjà remarquer qu'il y a une inégalité du nombre d'article dans le corpus selon les sites.

Par exemple le média Sputnik est bien plus représenté que les autres.

## Analyses sur les mots-clés

On crée une fonction qui nous renvoie les mots-clés et leurs nombres d'itérations pour un corpus donné.

On peut passer directement par ["metadata-all"]["fr"]["all"]["kws"] pour cette fonction.

In [59]:
def getKeywords(corpus, limit):
    """
    corpus (str array) : les fichiers du corpus
    limit (int) : pour prendre les x mots les plus communs
    """
    keywords = {}
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        kws = data["metadata-all"]["fr"]["all"]["kws"]
        
        for kw,iters in kws.items():
            if(kw in keywords):
                keywords[kw] += iters
            else:
                keywords[kw] = iters
    #Sort
    keywords = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
    
    # Mise en forme pour plotly
    commonWords = []
    for f,s in keywords:
        commonWords.append({"keyword": f, "iteration": s})

    # On prend seulement les x mots les plus communs (selon le param limit)
    commonWords = commonWords[0:limit]
    fig = px.bar(commonWords, x='keyword', y='iteration')
    fig.show()

On crée un bar chart des 100 mots-clés les plus communs du corpus.

In [54]:
getKeywords(corpusFiles, 100)

### Analyses plus poussées sur les mots-clés

#### Voir les liens entre les mots-clés

On crée une fonction qui donne pour un corpus donné et un mot-clé donné les mots-clés relatifs au mot-clé choisi.

Ici on est obligé de faire plusieurs boucles for imbriquées car nous devons parcourir article par article

In [57]:
def showLinkedKws(corpus, pickedKw, limit):
    linkedKws = {}
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for kw,iters in kws.items():
                                if(kw != pickedKw):
                                    if(kw in linkedKws):
                                        linkedKws[kw] += 1
                                    else:
                                        linkedKws[kw] = 1
    
    #Sort
    linkedKws = sorted(linkedKws.items(), key=lambda x:x[1], reverse=True)
    commonLinkedWords = []

    for f,s in linkedKws:
         commonLinkedWords.append({"keyword": f, "iteration": s})

    commonLinkedWords = commonLinkedWords[0:limit]
    fig = px.bar(commonLinkedWords, x='keyword', y='iteration', title=pickedKw)
    fig.show()

Un exemple avec le mot-clé "juif" ( mot totalement choisi au hasard :) )

In [58]:
pickedKeyword = "juif"

commonLinkedWords = showLinkedKws(corpusFiles, pickedKeyword, 100)


On peut aussi voir les personnalités en lien avec un mot-clef

In [5]:
def showLinkedPeople(corpus, pickedKw, limit):
    linkedPeople = {}
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]

                        if(pickedKw in kws):
                            for p,iter in people.items():
                                if(p in linkedPeople):
                                    linkedPeople[p] += 1
                                else:
                                    linkedPeople[p] = 1
            #Sort
    linkedPeople = sorted(linkedPeople.items(), key=lambda x:x[1], reverse=True)
    commonLinkedPeople = []

    for f,s in linkedPeople:
        commonLinkedPeople.append({"person": f, "iteration": s})

    commonLinkedPeople = commonLinkedPeople[0:limit]
    fig = px.bar(commonLinkedPeople, x='person', y='iteration',title=pickedKeyword)
    fig.show()

In [6]:
pickedKeyword = "juif"

showLinkedPeople(corpusFiles, pickedKeyword, 100)

Avec plusieurs keywords on a souvent le nom de "Alain Soral" qui revient.

On crée une nouvelle fonction qui nous donne aussi le site.

In [25]:
def showLinkedPeopleWithSite(corpus, pickedKw, limit):
    linkedPeopleWithSite = []
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        siteName = f.name.split(".")[1]
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:
                        people = article["per"]
                        kws = article["kws"]
                        if(pickedKw in kws):
                            for p,iter in people.items():
                                # Tester si la personne associé au site existe déjà
                                isNew = True
                                for e in linkedPeopleWithSite:
                                    if e["person"] == p:
                                        e["iteration"] += iter
                                        dicts = e["iterationWithSite"]
                                        if siteName in dicts:
                                            dicts[siteName] += iter
                                        else:
                                            dicts[siteName] = iter
                                        isNew = False
                                        break
                                if(isNew):
                                    linkedPeopleWithSite.append({"person": p, "iteration": iter, "iterationWithSite": {siteName: iter}})

    linkedPeopleWithSite = sorted(linkedPeopleWithSite, key=lambda x:x['iteration'], reverse=True)

    whatsup = []
    for e in linkedPeopleWithSite:
        person = e["person"]
        for site, iteration in e["iterationWithSite"].items():
           whatsup.append({"person": person, "iteration": iteration, "site": site})

    commonLinkedPeopleWithSite = whatsup[0:limit]

    print("Dataframe")
    commonLinkedPeopleWithSite = pd.DataFrame(commonLinkedPeopleWithSite)

    print(commonLinkedPeopleWithSite)

    # commonLinkedPeopleWithSite = whatsup

    #print(commonLinkedPeopleWithSite)
    
    fig = px.bar(commonLinkedPeopleWithSite, x='person', y='iteration', title=pickedKeyword)
    fig.update_layout(yaxis_categoryorder = 'total ascending')
    fig.show()

In [26]:
pickedKeyword = "israel"

showLinkedPeopleWithSite(corpusFiles, pickedKeyword, 100)

Dataframe
          person  iteration                     site
0          Trump        128  egaliteetreconciliation
1          Trump         17              sputniknews
2          Trump         29                  presstv
3   Donald Trump        101  egaliteetreconciliation
4   Donald Trump         24              sputniknews
..           ...        ...                      ...
95         Orbán          8  egaliteetreconciliation
96    Mme Winter          8  egaliteetreconciliation
97       Zemmour          8  egaliteetreconciliation
98        Curtis          8  egaliteetreconciliation
99       Erdogan          2              sputniknews

[100 rows x 3 columns]


On peut ainsi faire la même chose avec les organisations, localisations, etc.

On peut ainsi regrouper les mots-clés, personnes, organisations et localisations comme des "sujets" en lien avec un autre sujet.

On crée une fonction permettant de faire ceci tout en nous donnant le site qui y correspond

In [103]:
def showLinkedTopics(corpus, pickedTopic, limit):
    linkedTopics = []
    for file in corpus:
        f = open(file, 'r', encoding='utf-8')
        data = json.loads(f.read())
        f.close()
        siteName = f.name.split(".")[1]
        print(siteName + " :")
        years = data["data-all"].keys()
        for year in years:
            months = data["data-all"][year].keys()
            for month in months:
                days = data["data-all"][year][month].keys()
                for day in days:
                    articles = data["data-all"][year][month][day]
                    for article in articles:

                        people = article["per"]
                        kws = article["kws"]
                        locs = article["loc"]
                        orgs = article["org"]

                        topicsArticle = people | kws | locs | orgs

                        #print("aticle : " + str(len(topicsArticle)))

                        if(pickedTopic in topicsArticle):
                            del topicsArticle[pickedTopic]
                            for t,iter in topicsArticle.items():
                                # Tester si les topics associés son déjà dans la liste
                                isNew = True
                                for e in linkedTopics:
                                    if e["topic"] == t :#and e["site"] == siteName:
                                        e["iteration"] += iter
                                        isNew = False
                                        break
                                if(isNew):
                                    linkedTopics.append({"topic": t, "iteration": iter})
    #BubbleSort
    good = False

    while(not good):
        good = True
        for i in range(0, len(linkedTopics) - 1):
                iter1 = linkedTopics[i]["iteration"]
                iter2 = linkedTopics[i+1]["iteration"]
                if(iter2 > iter1):
                    linkedTopics[i], linkedTopics[i+1] = linkedTopics[i+1], linkedTopics[i]
                    good = False

    print(len(linkedTopics))
    commonLinkedTopics = linkedTopics[0:limit]

    
    fig = px.bar(commonLinkedTopics, x='topic', y='iteration', title=pickedKeyword)
    fig.show()  


In [104]:
testCorpus = ['topaz-data732--mali--www.egaliteetreconciliation.fr--20190101--20211231.json', 
'topaz-data732--mali--www.fdesouche.com--20190101--20211231.json',
'topaz-data732--mali--fr.sputniknews.africa--20190101--20211231.json',
'topaz-data732--mali--french.presstv.ir--20190101--20211231.json']

showLinkedTopics(testCorpus, "juif", 50)

egaliteetreconciliation :
fdesouche :
sputniknews :
presstv :
4780
